In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Config, AdamW
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('HooshvareLab/gpt2-fa')

#PoetryDataset class
class PoetryDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_text = self.df.iloc[idx]["input"]
        output_text = self.df.iloc[idx]["poem"]
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True)
        output_ids = self.tokenizer.encode(output_text, add_special_tokens=True)

        input_ids += [0] * (self.max_length - len(input_ids))
        output_ids += [0] * (self.max_length - len(output_ids))

        return torch.tensor(input_ids), torch.tensor(output_ids)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ferdousi.txt to ferdousi.txt


In [ ]:
import re
def extract_beyts(text):
    beyts = re.findall(r'([^\n]+)\n([^\n]+)', text)
    return beyts

#loading the poetry collection
with open("ferdousi.txt", "r", encoding="utf-8") as file:
    ferdowsi_poem = file.read()

beyts = extract_beyts(ferdowsi_poem)
beyts = beyts[1:]

#separating input and output
inputs = []
outputs = []
for i in range(len(beyts)):
    if i == 0:
        continue
    inputs.append(beyts[i-1])
    outputs.append(beyts[i])

df = pd.DataFrame({"input": inputs, "poem": outputs})

#separaring the dataset into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

#considering the maximum sequence length = 50
max_length = 50

Train shape: (39685, 2)
Test shape: (9922, 2)


In [ ]:
#creating train and test datasets by tokenizer
train_dataset = PoetryDataset(train_df, tokenizer, max_length)
test_dataset = PoetryDataset(test_df, tokenizer, max_length)

#creating batch size and data loaders
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#loading pre-trained model
model_name_or_path = "HooshvareLab/gpt2-fa"
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4)

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, labels=labels)
        logits = outputs.logits

        # Flatten the logits and labels
        logits = logits.view(-1, logits.shape[-1])
        labels = labels.view(-1)

        # Calculate the loss and perform backpropagation
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Avg. Training Loss: {avg_train_loss}")

Epoch 1/5: 100%|██████████| 1241/1241 [08:22<00:00,  2.47it/s]


Epoch 1 - Avg. Training Loss: 0.02741828220701303


Epoch 2/5: 100%|██████████| 1241/1241 [08:23<00:00,  2.46it/s]


Epoch 2 - Avg. Training Loss: 2.576959877292801e-05


Epoch 3/5: 100%|██████████| 1241/1241 [08:23<00:00,  2.47it/s]


Epoch 3 - Avg. Training Loss: 6.037133655169359e-06


Epoch 4/5: 100%|██████████| 1241/1241 [08:23<00:00,  2.46it/s]


Epoch 4 - Avg. Training Loss: 3.0795825488094183e-06


Epoch 5/5: 100%|██████████| 1241/1241 [08:23<00:00,  2.46it/s]

Epoch 5 - Avg. Training Loss: 1.960193125126438e-06


In [ ]:
#evaluation loop
model.eval()
eval_loss = 0.0
total_tokens = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        outputs = model(input_ids, labels=labels)
        logits = outputs.logits

        logits = logits.view(-1, logits.shape[-1])
        labels = labels.view(-1)

        loss = loss_fn(logits, labels)
        eval_loss += loss.item()

        total_tokens += labels.numel()

avg_eval_loss = eval_loss / len(test_dataloader)
perplexity = torch.exp(torch.tensor(avg_eval_loss))

print(f"Avg. Evaluation Loss: {avg_eval_loss}")
print(f"Perplexity: {perplexity}")


Evaluating: 100%|██████████| 311/311 [00:42<00:00,  7.34it/s]

Avg. Evaluation Loss: 2.777576355583733e-07
Perplexity: 1.000000238418579


In [ ]:
def generate_poetry(model, tokenizer, prompt_text, max_length=50, num_return_sequences=1):
    model.eval()
    input_ids = tokenizer.encode(prompt_text, add_special_tokens=True, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_poems = []

    for output in outputs:
        poem = tokenizer.decode(output, skip_special_tokens=True)
        generated_poems.append(poem)

    return generated_poems

prompt = "در دل صبح"

generated_poems = generate_poetry(model, tokenizer, prompt)

for poem in generated_poems:
    print(poem)

در دل صبح یک دست لباس می‌چینین و برمی گردین و به خواب میرید. با اینکه شب‌ها به فکر خواب هستین و بدن خواب نداره ولی یهو به خودتون میایین میبینی که دارید تو خواب از خواب بیدار می‌شین و
